In [1]:
import pandas as pd

# Specify the path to your Excel file
excel_file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\routes_selected_new.xlsx'
survey_file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv'
# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)
df_survey = pd.read_csv(survey_file_path, encoding='utf-8', delimiter=',')

In [60]:
# Create a copy of the original DataFrame df
df_routes = df.copy()

In [61]:
import numpy as np

# Iterate through all columns
for column in df_routes.columns:
    df_routes[column] = df_routes[column].replace(', ', np.nan)

# Specify the list of columns to keep
selected_columns = ['ID_mosta', 'Okres',
                    'GPS_Bridge', 'S_New', 'E_New', 'Custom_coor']

# Filter the DataFrame to include only the selected columns
df_routes = df_routes[selected_columns]

In [62]:
# Remove spaces from 'GPS_Bridge', 'S_New', and 'E_New' columns
df_routes['GPS_Bridge'] = df_routes['GPS_Bridge'].str.replace(' ', '')
df_routes['S_New'] = df_routes['S_New'].str.replace(' ', '')
df_routes['E_New'] = df_routes['E_New'].str.replace(' ', '')

# Split the coordinates by ','
df_routes[['br_lat', 'br_long']
          ] = df_routes['GPS_Bridge'].str.split(',', expand=True)
df_routes[['start_lat', 'start_long']
          ] = df_routes['S_New'].str.split(',', expand=True)
df_routes[['end_lat', 'end_long']
          ] = df_routes['E_New'].str.split(',', expand=True)

# Convert the new columns to float (if needed)
df_routes[['br_lat', 'br_long', 'start_lat', 'start_long', 'end_lat', 'end_long']] = df_routes[[
    'br_lat', 'br_long', 'start_lat', 'start_long', 'end_lat', 'end_long']].astype(float)

# Drop the original columns
df_routes.drop(columns=['GPS_Bridge', 'S_New', 'E_New'], inplace=True)

In [63]:
from math import radians, cos, sin, asin, sqrt, pi


def get_hexagon_coordinates(row, radius):
    """
    Calculate the coordinates of a hexagon centered at a given latitude and longitude.

    Parameters:
        row (pandas.Series): A row from the DataFrame containing 'br_lat' and 'br_long' columns.
        radius (float): The radius in meters of the circle in which the hexagon is inscribed.

    Returns:
        list of tuples: A list of tuples containing (latitude, longitude) coordinates of the hexagon vertices.
    """
    coordinates = []
    center_lat = row['br_lat']
    center_lon = row['br_long']
    for i in range(6):
        angle = 2 * pi / 6 * i
        dx = radius * cos(angle)
        dy = radius * sin(angle)
        point_lat = center_lat + (180/pi) * (dy / 6378137)
        point_lon = center_lon + (180/pi) * \
            (dx / 6378137) / cos(center_lat * pi/180)
        coordinates.append((point_lat, point_lon))
    return coordinates


# Radius of the circle in meters in which hexagon is inscribed
radius = 20  # 10 meters radius in kilometers

# Create a new column 'Hexagon_Coordinates' in df_routes
df_routes['Hexagon_Coordinates'] = None  # Initialize with None values

# Iterate through each row in the DataFrame
for index, row in df_routes.iterrows():
    # Create hexagon coordinates for each row
    hexagon_coords = get_hexagon_coordinates(row, radius)
    hexagon_coords.append(hexagon_coords[0])

    # Convert hexagon coordinates from (latitude, longitude) tuples to [longitude, latitude] lists
    hexagon_coords_for_api = [[lon, lat] for lat, lon in hexagon_coords]

    # Assign the hexagon coordinates to the 'Hexagon_Coordinates' column
    df_routes.at[index, 'Hexagon_Coordinates'] = hexagon_coords_for_api

In [75]:
import json
import pandas as pd

# Define the generate_graphhopper_request function without the row_index argument


def generate_graphhopper_request(row: pd.Series, profile: str = "car", multiplier: int = 1) -> str:
    """
    Generate a JSON request for the GraphHopper API based on DataFrame row data.

    Parameters:
        row (pd.Series): A row from the DataFrame containing route data.
        profile (str, optional): The vehicle profile for routing (e.g., "car", "bike", "foot").
        multiplier (int, optional): Multiplier for custom model priority within a specific area.

    Returns:
        str: A JSON string representing the GraphHopper API request.
    """
    hexagon_coords = row['Hexagon_Coordinates']

    data = {
        "points": [
            [row['start_long'], row['start_lat']],
            [row['end_long'], row['end_lat']]
        ],
        "profile": profile,
        "points_encoded": False,
        "ch.disable": True,
        "custom_model": {
            "distance_influence": 15,
            "priority": [
                {
                    "if": "in_area1",
                    "multiply_by": multiplier
                }
            ],
            "speed": [],
            "areas": {
                "area1": {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [hexagon_coords]
                    },
                    "properties": {}
                }
            }
        }
    }
    # Convert the dictionary to a JSON string
    json_data = json.dumps(data, indent=2)
    return json_data

In [76]:
# Before assigning JSON strings, make sure the columns are of type 'object'
df_routes['car_req_1'] = df_routes['car_req_1'].astype(object)
df_routes['car_req_alt'] = df_routes['car_req_alt'].astype(object)

# Iterate through each bridge in the DataFrame
for index, row in df_routes.iterrows():
    # Generate JSON requests for the current bridge and add them directly to the DataFrame
    df_routes.at[index, 'car_req_1'] = generate_graphhopper_request(
        row, profile="car", multiplier=1)
    df_routes.at[index, 'car_req_alt'] = generate_graphhopper_request(
        row, profile="car", multiplier=0)

In [86]:
# Create empty columns 'car_resp_1' and 'car_resp_alt'
df_routes['car_resp_1'] = None
df_routes['car_resp_alt'] = None

In [115]:
import requests
import json


def call_graphhopper_api(df, row_index, api_key, verbose=False):
    api_endpoint = "https://graphhopper.com/api/1/route"
    headers = {'Content-Type': 'application/json'}

    # Convert request strings to JSON format
    def convert_to_json(request_str):
        try:
            return json.loads(request_str)
        except json.JSONDecodeError as e:
            if verbose:
                print(f"Error in converting string to JSON: {e}")
            return None

    # Function to send a request to the GraphHopper API
    def send_request(request_data):
        if request_data is None:
            if verbose:
                print("Invalid request data. Skipping request.")
            return None

        if verbose:
            print("Sending request to GraphHopper API...")
        response = requests.post(
            api_endpoint, headers=headers, json=request_data, params={'key': api_key})
        if response.status_code == 200:
            if verbose:
                print("Response received successfully.")
            return response.json()
        else:
            if verbose:
                print(
                    f"Failed to receive response: {response.status_code}, Response: {response.text}")
            return None

    # Sending requests and updating the DataFrame
    car_req_1_json = convert_to_json(df.iloc[row_index]["car_req_1"])
    car_req_alt_json = convert_to_json(df.iloc[row_index]["car_req_alt"])

    df.at[row_index, 'car_resp_1'] = send_request(car_req_1_json)
    df.at[row_index, 'car_resp_alt'] = send_request(car_req_alt_json)

    if verbose:
        print("Requests completed.")

In [117]:
import requests

# Define the GraphHopper API endpoint and API key
api_endpoint = "https://graphhopper.com/api/1/route"
# Replace with your actual API key
api_key = "f0b121e6-90d8-4151-b56d-25563fa4fbf9"

# Specify the row index you want to process
row_index_to_process = 0  # Change this to the desired row index

# Usage example:
call_graphhopper_api(df_routes, row_index_to_process,
                     api_key, verbose=True)

Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.


In [154]:
for i in range(126, 148):
    call_graphhopper_api(df_routes, i,
                         api_key, verbose=True)
    print(i)

Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
126
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
127
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
128
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
129
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
130
Sending request to GraphHopper API...
Response received successfully.
Sending request to GraphHopper API...
Response received successfully.
Requests completed.
131
Sending request 

In [157]:
df_routes.iloc[107]

ID_mosta                                                           M3011
Okres                                                       Bratislava I
Custom_coor                                                          NaN
br_lat                                                         48.162426
br_long                                                        17.097242
start_lat                                                      48.168682
start_long                                                     17.076438
end_lat                                                        48.159364
end_long                                                       17.104073
Hexagon_Coordinates    [[17.097511269343144, 48.162426001], [17.09737...
car_req_1              {\n  "points": [\n    [\n      17.076438,\n   ...
car_req_alt            {\n  "points": [\n    [\n      17.076438,\n   ...
car_resp_1             {'hints': {'visited_nodes.sum': 80, 'visited_n...
car_resp_alt                                       

In [107]:
type(df_routes.iloc[0]["car_req_1"])

str

In [158]:
import feather

# Save your DataFrame to a Feather file
feather.write_dataframe(df_routes, 'data_with_responses.feather')

In [161]:
# Load the DataFrame from the Feather file
loaded_df = feather.read_dataframe('data_with_responses.feather')